# Planning Pattern for AI Agents
## Introduction
This notebook shows how to implement a basic planning pattern for AI agents. Planning helps AI agents break down complex tasks into simpler steps and execute them in order

Think of planning like how you might plan a vacation:
1. First, you decide where to go
2. Then you check flights and hotels
3. Next you book everything
4. Finally you pack and prepare

Our AI agent will work the same way - simplifying or breaking big tasks into smaller, manageble steps.

## 1. First let's import what we need

In [2]:
import os
from openai import OpenAI
import json
from datetime import datetime

# initialize MaLLaM client
mallam_api_key = os.getenv("MALLAM_API_KEY")

client = OpenAI(
    base_url="https://api.mesolitica.com",
    api_key=mallam_api_key
)

## 2. Basic Planning Example
Let's start with a simple example: planning a birthday party. We'll ask the AI to break this task down into manageable steps

In [3]:
# Example task: Plan a birthday party
task = "Help me plan a birthday party for 10 people"

# create the planning prompt
planning_prompt = [
    {
        "role": "system",
        "content": """You are a helpful planning assisstant. When given a task:
        1. Break it down into clear, sequential steps
        2. For each step, include:
            - What needs to be done
            - Any important details or considerations
        3. Keep the plan simple and actionable
        Return the plan as a JSON array with 'steps' containing:
        steps:
        - step_number
        - description
        - estimated_time (in hours)"""
    },
    {
        "role": "user",
        "content": task
    }
]

# get the plan from MaLLaM
response = client.chat.completions.create(
    model="mallam-small",
    messages=planning_prompt,
    max_tokens=1024
)

# parse the response into a plan
plan = json.loads(response.choices[0].message.content)
print(plan)

print("\nGenerated Plan:")
for step in plan["steps"]:
    print(f"\nStep {step['step_number']}:")
    print(f"\nDescription: {step['description']}")
    print(f"\nEstimated time: {step['estimated_time']} hours")

{'steps': [{'step_number': 1, 'description': 'Determine the theme of the party', 'estimated_time': 1}, {'step_number': 2, 'description': 'Choose a venue or decide to have it at home', 'estimated_time': 1}, {'step_number': 3, 'description': 'Make a guest list (10 people)', 'estimated_time': 0.5}, {'step_number': 4, 'description': 'Send out invitations', 'estimated_time': 1}, {'step_number': 5, 'description': 'Plan the menu and arrange for food and drinks', 'estimated_time': 2}, {'step_number': 6, 'description': 'Arrange for entertainment or activities', 'estimated_time': 1.5}, {'step_number': 7, 'description': 'Decorate the venue according to the theme', 'estimated_time': 2}, {'step_number': 8, 'description': 'Prepare party favors or gifts for guests', 'estimated_time': 1}, {'step_number': 9, 'description': 'Confirm final arrangements and ensure all is set for the day', 'estimated_time': 1}]}

Generated Plan:

Step 1:

Description: Determine the theme of the party

Estimated time: 1 hou

## 3. Plan Execution Tracking
Now let's add simple tracking to monitor the progress of our plan.

In [4]:
# create a dictionary to track progress
progress = {
    "total_steps": len(plan["steps"]),
    "completed_steps": 0,
    "current_step": 1,
    "start_time": datetime.now().isoformat(),
    "step_status": {str(step["step_number"]): "pending" for step in plan["steps"]}
}

# example: Mark a step as complete
step_to_complete = "1" # complete the first step
progress["step_status"][step_to_complete] = "completed"
progress["completed_steps"] += 1
progress["current_step"] = 2

print("\nPlan Progress:")
print(f"Completed: {progress['completed_steps']}/{progress['total_steps']} steps")
for step_num, status in progress["step_status"].items():
    print(f"Step {step_num}: {status}")


Plan Progress:
Completed: 1/9 steps
Step 1: completed
Step 2: pending
Step 3: pending
Step 4: pending
Step 5: pending
Step 6: pending
Step 7: pending
Step 8: pending
Step 9: pending


## 4. Real-World Example: Writing a Blog Post
Let's try a more practical example that many developers might encounter

In [5]:
# Task: write a technical blog post about Agentic AI for beginners
blog_task = "Create a plan for writing a beginner-friendly blog post about Agentic AI basics"

blog_planning_prompt = [
    {
        "role": "system",
        "content": """You are a technical writing assistant. Create a detailed plan for writing a blog post.
        Return the plan as a JSON with:
        - title: suggested blog post title
        - target_audience: who the post is for
        - steps: array of writing steps with:
            - step_number
            - section_title
            - key_points (array)
            - estimated_time (in hours)"""
    },
    {
        "role": "user",
        "content": blog_task
    }
]

# get the blog writing plan
blog_response = client.chat.completions.create(
    model="mallam-small",
    messages=blog_planning_prompt,
    max_tokens=1024
)

# parse and display the blog plan
blog_plan = json.loads(blog_response.choices[0].message.content)
print(blog_plan)

print(f"\nBlog Post Plan: {blog_plan['title']}")
print(f"Target Audience: {blog_plan['target_audience']}")
print("\nWriting Steps:")
for step in blog_plan["steps"]:
    print(f"\nSection {step['step_number']}: {step['section_title']}")
    print("Key Points:")
    for point in step['key_points']:
        print(f"- {point}")
    print(f"Estimated time: {step['estimated_time']} hours")


{'title': 'Introduction to Agentic AI for Beginners', 'target_audience': 'Beginners interested in understanding the basics of Agentic AI, including its definition, applications, and potential impacts on society.', 'steps': [{'step_number': 1, 'section_title': 'Define Agentic AI', 'key_points': ['Explain what Agentic AI is and how it differs from traditional AI.', 'Discuss the concept of agency in AI and its significance.', 'Provide examples of agentic systems and their characteristics.'], 'estimated_time': 2}, {'step_number': 2, 'section_title': 'Explore Applications of Agentic AI', 'key_points': ['Describe various domains where Agentic AI is being applied, such as robotics, autonomous vehicles, and healthcare.', 'Discuss case studies or real-world examples to illustrate these applications.', 'Highlight the benefits and potential advantages of using Agentic AI in these areas.'], 'estimated_time': 3}, {'step_number': 3, 'section_title': 'Discuss Ethical Considerations', 'key_points': ['

## 5. Plan Adaptation
Sometimes plans need to change. Here's how we can modify our plan based on new information

In [13]:
new_constraint = "We need to complete the blog post in half the original time"

adaptation_prompt = [
    {
        "role": "system",
        "content": """You are a planning assistant. Given a plan and new constraints:
        1. Analyze the current plan
        2. Suggest modifications to meet new constraints
        3. Return the modified plan as a JSON with:
        - title: suggested blog post title
        - target_audience: who the post is for
        - steps: array of writing steps with:
            - step_number
            - section_title
            - key_points (array)
            - estimated_time (in hours):
        Keep the essential elements while adjusting scope/time."""
    },
    {
        "role": "user",
        "content": f"""Original plan: {json.dumps(blog_plan)}
        New constraint: {new_constraint}"""
    }
]

# get the updated plan
adaptation_response = client.chat.completions.create(
    model="mallam-small",
    messages=adaptation_prompt,
    max_tokens=1024
)

# Parse and show the adapted plan
adapted_plan = json.loads(adaptation_response.choices[0].message.content)
print(adapted_plan)

print("\nAdapted Plan:")
print(f"Original total time: {sum(step['estimated_time'] for step in blog_plan['steps'])} hours")
print(f"Adapted total time: {sum(step['estimated_time'] for step in adapted_plan['steps'])} hours")

# Show what changed
print("\nChanges made:")
for orig, adapted in zip(blog_plan["steps"], adapted_plan["steps"]):
    if orig != adapted:
        print(f"\nSection: {adapted['section_title']}")
        print(f"Original time: {orig['estimated_time']} hours")
        print(f"Adapted time: {adapted['estimated_time']} hours")

{'title': 'Understanding Agentic AI: Basics and Impacts', 'target_audience': 'Beginners looking to grasp the core principles of Agentic AI and its societal implications.', 'steps': [{'step_number': 1, 'section_title': 'Defining Agentic AI', 'key_points': ["Explain Agentic AI's essence and distinction from traditional AI.", 'Discuss the concept of agency in AI and its importance.', 'Provide examples of agentic systems and their features.'], 'estimated_time': 1}, {'step_number': 2, 'section_title': 'Applications Overview', 'key_points': ['Outline key areas where Agentic AI is utilized, like robotics and healthcare.', 'Mention a few real-world applications to illustrate.', 'Highlight the benefits and advantages of Agentic AI in these fields.'], 'estimated_time': 1.5}, {'step_number': 3, 'section_title': 'Ethical Considerations', 'key_points': ['Address ethical issues linked to Agentic AI, such as privacy and accountability.', 'Look into current regulations governing Agentic AI.', 'Prompt 